# Homework: NYC flights 2013 Analysis
 

# Install packages and Load library

In [4]:
install.packages("nycflights13")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [5]:
library(dplyr)

In [6]:
library(nycflights13)

# preview data 

In [7]:
glimpse(flights)
glimpse(airlines)
glimpse(airports)

Rows: 336,776
Columns: 19
$ year           <int> 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2…
$ month          <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ day            <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ dep_time       <int> 517, 533, 542, 544, 554, 554, 555, 557, 557, 558, 558, …
$ sched_dep_time <int> 515, 529, 540, 545, 600, 558, 600, 600, 600, 600, 600, …
$ dep_delay      <dbl> 2, 4, 2, -1, -6, -4, -5, -3, -3, -2, -2, -2, -2, -2, -1…
$ arr_time       <int> 830, 850, 923, 1004, 812, 740, 913, 709, 838, 753, 849,…
$ sched_arr_time <int> 819, 830, 850, 1022, 837, 728, 854, 723, 846, 745, 851,…
$ arr_delay      <dbl> 11, 20, 33, -18, -25, 12, 19, -14, -8, 8, -2, -3, 7, -1…
$ carrier        <chr> "UA", "UA", "AA", "B6", "DL", "UA", "B6", "EV", "B6", "…
$ flight         <int> 1545, 1714, 1141, 725, 461, 1696, 507, 5708, 79, 301, 4…
$ tailnum        <chr> "N14228", "N24211", "N619AA", "N804JB", "N668DN", "N394…
$ origin      

# Q1: What carriers that flight did arrival delay on 08 July 2013?

In [8]:
carrier_delay <- flights  %>%
  filter(month == 7 & year == 2013 & day == 8) %>%
  filter(arr_delay > 0) %>%
  count(carrier) %>%
  arrange(desc(n)) %>%
  rename(n_flights_arr_delay = n) %>%
  left_join(airlines, by = "carrier")

carrier_delay %>%
  select(carrier, name, n_flights_arr_delay) %>%
  rename(carrier_name = name) 

carrier,carrier_name,n_flights_arr_delay
<chr>,<chr>,<int>
B6,JetBlue Airways,103
EV,ExpressJet Airlines Inc.,96
UA,United Air Lines Inc.,92
DL,Delta Air Lines Inc.,65
MQ,Envoy Air,50
AA,American Airlines Inc.,33
9E,Endeavor Air Inc.,32
US,US Airways Inc.,32
WN,Southwest Airlines Co.,23


# Q2: What is the route's longest and shortest distance?

In [9]:
## Create dataframes of airports for full name origin and dest.

airports_for_origin <- airports %>%
  select(faa,  name) %>%
  rename(origin = faa,
         origin_name = name)

airports_for_dest <- airports %>%
  select(faa,  name) %>%
  rename(dest = faa,
         dest_name = name)
  
## Long

long_route <- flights %>%
  select(origin, dest, distance) %>%
  left_join(airports_for_origin, by = "origin") %>%
  left_join(airports_for_dest, by = "dest") %>%
  arrange(desc(distance)) %>%
  head(1) 

long_route <- long_route %>%
  select(origin_name, dest_name, distance)

## Short

short_route <- flights %>%
  select(origin, dest, distance) %>%
  left_join(airports_for_origin, by = "origin") %>%
  left_join(airports_for_dest, by = "dest") %>%
  arrange(desc(distance)) %>%
  tail(1)

short_route <- short_route %>%
  select(origin_name, dest_name, distance)
  
  
## Bind_row dataframe

route <- list(short_route, long_route)
bind_rows(route)

origin_name,dest_name,distance
<chr>,<chr>,<dbl>
Newark Liberty Intl,La Guardia,17
John F Kennedy Intl,Honolulu Intl,4983


# Q3: What are carriers that depart and arrive on time in (not delayed) 2013?

In [10]:
carrier_delay <- flights  %>%
  filter(month == 7 & year == 2013 & day == 8) %>%
  filter(arr_delay > 0) %>%
  count(carrier) %>%
  arrange(desc(n)) %>%
  left_join(airlines, by = "carrier")

carrier_delay %>%
  select(carrier, name, n) %>%
  rename(carrier_full_name = name,
         n_flights = n) 
         

carrier,carrier_full_name,n_flights
<chr>,<chr>,<int>
B6,JetBlue Airways,103
EV,ExpressJet Airlines Inc.,96
UA,United Air Lines Inc.,92
DL,Delta Air Lines Inc.,65
MQ,Envoy Air,50
AA,American Airlines Inc.,33
9E,Endeavor Air Inc.,32
US,US Airways Inc.,32
WN,Southwest Airlines Co.,23


# Q4: What's carriers depart on time at 12.00 P.M. (not delayed) Jan 2013?

In [11]:
df <- flights %>%
  filter(month == 1,
         year == 2013,
         sched_dep_time == 1200,
         dep_delay == 0) %>%
  select(carrier, year, month, day, dep_time, dep_delay, arr_time, sched_arr_time, arr_delay)  %>%
  left_join(airlines, by = "carrier")

df %>%
    select(carrier, name, year, month, day, dep_time, dep_delay, arr_time, sched_arr_time, arr_delay) %>%
    rename(carrier_full_name = name)

carrier,carrier_full_name,year,month,day,dep_time,dep_delay,arr_time,sched_arr_time,arr_delay
<chr>,<chr>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>
US,US Airways Inc.,2013,1,1,1200,0,1408,1356,12
UA,United Air Lines Inc.,2013,1,2,1200,0,1513,1505,8
UA,United Air Lines Inc.,2013,1,5,1200,0,1517,1459,18
UA,United Air Lines Inc.,2013,1,6,1200,0,1514,1529,-15
US,US Airways Inc.,2013,1,7,1200,0,1352,1400,-8
DL,Delta Air Lines Inc.,2013,1,14,1200,0,1533,1529,4
MQ,Envoy Air,2013,1,17,1200,0,1320,1315,5
MQ,Envoy Air,2013,1,17,1200,0,1447,1430,17
MQ,Envoy Air,2013,1,18,1200,0,1331,1330,1


# Q5: Top 5 Destinations, most flights arriving on Christmas day?

In [12]:
most_arrive_dest <- flights %>%
  filter(month == 12,
         year == 2013,
         day == 25) %>%
  count(dest) %>%
  arrange(desc(n)) %>%
  left_join(airports, by = c("dest" = "faa" )) 

most_arrive_dest %>%
  select(dest, name, n) %>%
  rename(dest_full_name = name,
         n_flights = n) %>%
         head(5)

dest,dest_full_name,n_flights
<chr>,<chr>,<int>
MCO,Orlando Intl,41
FLL,Fort Lauderdale Hollywood Intl,39
ATL,Hartsfield Jackson Atlanta Intl,37
LAX,Los Angeles Intl,36
CLT,Charlotte Douglas Intl,32


# RPostgreSQL

# Install package and Load library

In [8]:
install.packages("RPostgreSQL")

Warning message in install.packages("RPostgreSQL"):
“installation of package ‘RPostgreSQL’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [9]:
library(RPostgreSQL)

ERROR: ERROR: Error in library(RPostgreSQL): there is no package called ‘RPostgreSQL’


# connect server

In [10]:
con <- dbConnect(
  PostgreSQL(),
  host = "arjuna.db.elephantsql.com",
  dbname = "osrwijbi",
  port = 5432,
  user = "osrwijbi",
  password = "UZ4k6gUoBSJwhc8wT1k0xtyKXSmBNHBO"
)

ERROR: ERROR: Error in dbConnect(PostgreSQL(), host = "arjuna.db.elephantsql.com", dbname = "osrwijbi", : could not find function "dbConnect"


## Check List Table in server

In [ ]:
dbListTables(con)

## Create dataframe

In [ ]:
friends_football <- data.frame(
  id = 1:5,
  friends_names = c("Elsa", "Tham", "Poom", "Sotus", "Tar"),
  fav_team_football = c("ManU", "Arsenal", "Arsenal", "Liverpool", "Liverpool")
)

## Save data

In [ ]:
dbWriteTable(con, "favorit_team_my_friends", friends_football)

## Get data from sever

In [ ]:
df <- dbGetQuery(con, "SELECT * FROM favorit_team_my_friends")

## Close connection

In [ ]:
dbDisconnect(con)